In [1]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2
import mysql.connector

## Conectar a AWS

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [3]:
aws_rds_conn = boto3.client('rds', aws_access_key_id = config.get('IAM','ACCESS_KEY'),
                            aws_secret_access_key = config.get('IAM','SECRET_KEY'),
                            region_name = 'us-east-2')

In [4]:
rds_intances_ids = []

aws_response = aws_rds_conn.describe_db_instances() # retorna un objeto [diccionario] iterable de las instancias

for response in aws_response['DBInstances']:
    rds_intances_ids.append(response['DBInstanceIdentifier'])

print(f'Instancias disponibles: {rds_intances_ids}')

Instancias disponibles: ['dbdim', 'dbtienda', 'sakila-db-pg-v']


In [5]:
try:
    response = aws_rds_conn.create_db_instance(
                                DBInstanceIdentifier=config.get('DBTIENDA','DB_INSTANCE_ID'),
                                DBName=config.get('DBTIENDA','DB_NAME'),
                                MasterUsername=config.get('DBTIENDA','DB_USERNAME'),
                                MasterUserPassword=config.get('DBTIENDA','DB_PASSWORD'),
                                Port=int(config.get('DBTIENDA','DB_PORT')),
                                DBInstanceClass='db.t3.micro',
                                Engine=config.get('DBTIENDA','DB_ENGINE'),
                                PubliclyAccessible=True,
                                AllocatedStorage=20,
                                VpcSecurityGroupIds=[config.get('VPC','SECURITY_GROUP')],
                            )
    print('Base de Datos creada exitosamente!!!')
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!!!',ex)

La instancia ya existe


### Access point

In [6]:
try:  
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('DBTIENDA','DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error!!!!',ex)

dbtienda.cp6geq8ycm59.us-east-2.rds.amazonaws.com


### Conexion a BD transaccional en mysql y creacion de tablas

In [99]:
import ddl_transacdb # py donde se encuentra el ddl
#ddl_transdb.ddl

In [129]:
cnx = mysql.connector.connect(
    host=RDS_HOSTNAME,
    user=config.get('DBTIENDA','DB_USERNAME'),
    password=config.get('DBTIENDA','DB_PASSWORD'),
    database=config.get('DBTIENDA','DB_NAME')
)
# Cursor
cursor = cnx.cursor()
cursor.execute(ddl_transdb.ddl) # creacion de tablas
cnx.close()
cursor.close()

True

### Ingreso de datos

In [130]:
from faker import Faker
import random

In [131]:
cnx = mysql.connector.connect(
    host=RDS_HOSTNAME,
    user=config.get('DBTIENDA','DB_USERNAME'),
    password=config.get('DBTIENDA','DB_PASSWORD'),
    database=config.get('DBTIENDA','DB_NAME')
)
# Cursor
cursor = cnx.cursor()

In [132]:
# Crear un objeto Faker
faker = Faker()

# TABLA CATEGORIAS
categorias = ['ALIMENTOS','JUGUETES','HOGAR','CALZADO','ROPA','DEPORTES']
for categ in categorias:  # insertamos 6 categorias
    nombre_categoria = categ
    descripcion_categoria = faker.sentence()
    estado_categoria = random.choices([0, 1], weights=[0.2,0.8])[0]  # Colocamos solo dos tipos de categorias (0 o 1)

    # Query de inserción para la tabla "categoria"
    sql_categoria = "INSERT INTO categoria (nombre, descripcion, estado) VALUES (%s, %s, %s)"
    val_categoria = (nombre_categoria, descripcion_categoria, estado_categoria)

    # Ejecutar la consulta para la tabla "categoria"
    cursor.execute(sql_categoria, val_categoria)

In [133]:
# TABLA ARTICULO
for _ in range(100):  #Insertamos 100 datos
    idcategoria = random.randint(1, 6)  # Le colocamos hasta el 5 porque solo colocamos 5 categorias
    codigo = faker.uuid4()[:6]  # Generamos un codigo uuid
    nombre_articulo = faker.word()  # Le colocamos nombre
    precio_venta = round(random.uniform(100, 1000), 2)  # Le colocamos un precio entre 100 y mil que tenga 2 decimales como maximo
    stock = random.randint(1, 100)
    descripcion_articulo = faker.text()  # Le colocamos la descripcion del articulo
    imagen = faker.file_name(category='image', extension='jpg')  # Agregamos una imagen con extension jpg
    estado_articulo = random.choice([0, 1])  # Estado aleatorio (0 o 1)

    # Insertamos los datos a la base de datos
    sql_articulo = "INSERT INTO articulo (idcategoria, codigo, nombre, precio_venta, stock, descripcion, imagen, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val_articulo = (idcategoria, codigo, nombre_articulo, precio_venta, stock, descripcion_articulo, imagen, estado_articulo)

    # Ejecutar la consulta para la tabla "articulo"
    cursor.execute(sql_articulo, val_articulo)

In [134]:
# TABLA PERSONA
for _ in range(10):  # Le dejamos los 10 como se establecio en la tabla ingreso (para no crear conflictos)
    tipo_persona = faker.random_element(elements=('PERSONA', 'EMPRESA'))  # Tipo de persona ficticio
    nombre_persona = faker.name()  # Nombre de persona ficticio
    tipo_documento = faker.random_element(elements=('DPI', 'PASAPORTE'))  # Tipo de documento ficticio
    num_documento = faker.unique.random_number(digits=13)  # DEJAMOS EL NUMERO COMO APARECE EN EL DPI
    direccion_persona = faker.address()
    telefono_persona = faker.phone_number()
    email_persona = faker.email()

    # Query de inserción para la tabla "persona"
    sql_persona = "INSERT INTO persona (tipo_persona, nombre, tipo_documento, num_documento, direccion, telefono, email) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val_persona = (tipo_persona, nombre_persona, tipo_documento, num_documento, direccion_persona, telefono_persona, email_persona)

    # Ejecutar la consulta para la tabla "persona"
    cursor.execute(sql_persona, val_persona)

In [135]:
# Tabla ROL

roles = ['Cajero','Jefe Tienda','Gerente']
for rol in roles:  # Lo dejamos como las categorias que colocamos a un inicio para no generar inconvenientes
    nombre_rol = rol
    descripcion_rol = faker.sentence()
    estado_rol = 1  # Colocamos solo dos tipos de estados como lo hicimos inicialmente

    # Query de inserción para la tabla "rol"
    sql_rol = "INSERT INTO rol (nombre, descripcion, estado) VALUES (%s, %s, %s)"
    val_rol = (nombre_rol, descripcion_rol, estado_rol)

    # Ejecutar la consulta para la tabla "rol"
    cursor.execute(sql_rol, val_rol)

In [136]:
# tabla "usuario"
for _ in range(10):  # Agregamos solo 10 datos para no generar inconvenientes
    idrol_usuario = random.randint(1, 3)  # que solo esten del 1 al 3 para que no genere inconvenientes con las llaves foraneas
    nombre_usuario = faker.name()  # Nombre de usuario ficticio
    tipo_documento_usuario = faker.random_element(elements=('DPI', 'PASAPORTE'))  # LO DEJAMOS COMO LA TABLA ANTERIOR
    num_documento_usuario = faker.unique.random_number(digits=13)
    direccion_usuario = faker.address()
    telefono_usuario = faker.phone_number()
    email_usuario = faker.email()
    clave_usuario = faker.password()
    estado_usuario = random.choices([0, 1], weights=[0.2,0.8])[0]
    
    # Query de inserción para la tabla "persona"
    sql_usuario = "INSERT INTO usuario (idrol, nombre, tipo_documento, num_documento, direccion, telefono, email, clave, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val_usuario = (idrol_usuario, nombre_usuario, tipo_documento_usuario, num_documento_usuario, direccion_usuario, telefono_usuario, email_usuario, clave_usuario, estado_usuario)

    # Ejecutar la consulta para la tabla "persona"
    cursor.execute(sql_usuario, val_usuario)    


In [137]:
#TABLA VENTA
for _ in range(100):
    idcliente_venta = random.randint(1, 10)  # Lo dejamos como lo colocamos a un inicio
    idusuario_venta = random.randint(1, 10)  # Lo dejamos como lo colocamos a un inicio
    tipo_comprobante_venta = faker.random_element(elements=('Factura', 'Nota de Credito'))  # Tipo de comprobante ficticio
    serie_comprobante_venta = faker.random_number(digits=5)  # Lo dejamos con la serie que declaramos arriba
    num_comprabante_venta = faker.random_number(digits=10)  # Lo dejamos como el numero que declaramos arribla
    fecha_venta = faker.date_time_between(start_date='-2y', end_date='now')  # Que la fecha sea de dos años en adelante
    #impuesto_venta = round(random.uniform(0, 120), 2)  # Le colocamos el maximo de impuesto
    #total_venta = round(random.uniform(100, 1000), 2)  # Total aleatorio
    estado_venta = faker.random_element(elements=('Pendiente', 'Pagado'))  # Estado de venta ficticio

    # Query de inserción para la tabla "venta"
    sql_venta = "INSERT INTO venta (idcliente, idusuario, tipo_comprobante, serie_comprobante, num_comprabante, fecha, impuesto, total, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val_venta = (idcliente_venta, idusuario_venta, tipo_comprobante_venta, serie_comprobante_venta, num_comprabante_venta, fecha_venta, impuesto_venta, total_venta, estado_venta)

    # Ejecutar la consulta para la tabla "venta"
    cursor.execute(sql_venta, val_venta)


In [138]:
# TABLA INGRESO
for _ in range(100):  # Inserrtamos 100 registros a la tabla
    idproveedor = random.randint(1, 10)  # ID de proveedor ficticio
    idusuario = random.randint(1, 10)  # ID de usuario ficticio
    tipo_comprobante = faker.random_element(elements=('Factura', 'Nota de Credito'))  # Le colocamos los tipos de documentos
    serie_comprobante = faker.random_number(digits=5)
    num_comprobante = faker.random_number(digits=10)
    fecha = faker.date_time_between(start_date='-2y', end_date='now')  # Le colocamos que los datos se encuentren dentro del 2022 al 2024
    #impuesto = round(random.uniform(0, 120), 2) # Le colocamos el maximo del iVA
    #total = round(random.uniform(100, 1000), 2) # Lo dejamos como en articulo
    estado_ingreso = faker.random_element(elements=('En Bodega', 'En Tienda'))  # Estado de ingreso ficticio

    # Query de inserción para la tabla "ingreso"
    sql_ingreso = "INSERT INTO ingreso (idproveedor, idusuario, tipo_comprobante, serie_comprobante, num_comprobante, fecha, impuesto, total, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val_ingreso = (idproveedor, idusuario, tipo_comprobante, serie_comprobante, num_comprobante, fecha, impuesto, total, estado_ingreso)

    # Ejecutar la consulta para la tabla "ingreso"
    cursor.execute(sql_ingreso, val_ingreso)


In [139]:
#TABLA DETALLE INGRESO
for idingr in range(1,101):  # Lo dejamos como el anterior
    idingreso_detalle = idingr # 1 a 100 registros
    for registro in range(random.randint(1,4)):    
        idarticulo_detalle = random.randint(1, 100)
        cantidad_detalle = random.randint(1, 20)  # Cantidad ficticia
        precio_detalle = round(random.uniform(10, 1000), 2)  # Precio ficticio

        # Query de inserción para la tabla "detalle_ingreso"
        sql_detalle_ingreso = "INSERT INTO detalle_ingreso (idingreso, idarticulo, cantidad, precio) VALUES (%s, %s, %s, %s)"
        val_detalle_ingreso = (idingreso_detalle, idarticulo_detalle, cantidad_detalle, precio_detalle)

        # Ejecutar la consulta para la tabla "detalle_ingreso"
        cursor.execute(sql_detalle_ingreso, val_detalle_ingreso)

# Actualizar monto total en tabla Ingreso
cursor.execute('''UPDATE ingreso SET total = (SELECT SUM(cantidad * precio) FROM detalle_ingreso WHERE detalle_ingreso.idingreso = ingreso.idingreso) WHERE ingreso.idingreso IN (SELECT idingreso FROM detalle_ingreso);''')


In [140]:
# TABLA DETALLE VENTA
for idvta in range(1,101):  # Insertar 10 registros de ejemplo
    idventa_detalle = idvta
    for regvta in range(random.randint(1,4)):
        idarticulo_detalle_venta = random.randint(1, 100)
        cantidad_detalle_venta = random.randint(1, 20)
        precio_detalle_venta = round(random.uniform(100, 1000), 2)
        descuento_detalle_venta = round(random.uniform(0, 20), 2)

        # Query de inserción para la tabla "detalle_venta"
        sql_detalle_venta = "INSERT INTO detalle_venta (idventa, idarticulo, cantidad, precio, descuento) VALUES (%s, %s, %s, %s, %s)"
        val_detalle_venta = (idventa_detalle, idarticulo_detalle_venta, cantidad_detalle_venta, precio_detalle_venta, descuento_detalle_venta)

        # Ejecutar la consulta para la tabla "detalle_venta"
        cursor.execute(sql_detalle_venta, val_detalle_venta)

# actualizar monto total en tabla Venta
cursor.execute('''UPDATE venta SET total = (SELECT SUM(cantidad * (precio-descuento)) FROM detalle_venta WHERE detalle_venta.idventa = venta.idventa) WHERE venta.idventa IN (SELECT idventa FROM detalle_venta);''')
cnx.commit()

In [141]:
cnx.close()
cursor.close()

True